In [152]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
import pandas as pd

import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/codyoshiro/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/codyoshiro/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/codyoshiro/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [153]:
data = pd.read_csv("YouTube.csv")

In [154]:
data

,ID,Video Title,Views,Likes,Dislikes,Transcript,class
0,hxwpkM5w3Cc,I Got Hunted By The FBI,42724724,2005151,24646,- I tied up an FBI agent.\n(words pop)If he st...,1
1,81mXAkFEkMs,10 Real Life Giants You Won't Believe Exist,6890718,15222,2858,hey what's happening y'all welcome backto anot...,1
2,9NQQKNAwaPk,10 Mythical CREATURES That Actually Existed,2089601,46750,1954,I used to taste your daddy who doesn'tlove a g...,1
3,Y_PQrPRolvM,Real Giants Caught On Camera!,518170,4033,271,Hi!\nWho said that giants are imaginary and on...,1
4,ZZtYAOpHVYk,10 Real Life GIANTS,9244943,64025,6508,[Music]hey guys what's up it's charlie here an...,1
...,...,...,...,...,...,...,...
150,x_qSdnjhbw4,Building an UNBREAKABLE TESSERACT CUBE!,1687124,81479,1474,"Of course, the main priority\nis to make the b...",0
151,PugcxQYJGjs,Lanky Kong listens to DK Rap for the first time,839745,102699,567,"- Yo! What up, guys?Lanky Kong here with\nanot...",0
152,cigmdYsgg_4,Can We Become American Ninja Warriors (Daequan...,819705,68459,402,all rightwhat's up guys welcome to the doom an...,0
153,s7MTM4BKZ_E,Why Anecdotes Trump Data,980055,36060,1347,Some critics of the TV show Mythbusters claimt...,0


In [155]:
data["class"].value_counts()

0    84
1    71
Name: class, dtype: int64

In [156]:
def clean_and_preprocess(text):
    # 1. Convert text to lowercase
    text = text.lower()

    # 2. Remove special characters, numbers, and punctuation
    text = re.sub(r'[^a-zA-Z\s]', '', text)

    # 3. Tokenize the text into words
    tokens = word_tokenize(text)

    # 4. Remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]

    # 5. Lemmatize words (convert words to their base form)
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]

    # 6. Join the preprocessed tokens back into a clean text
    cleaned_text = ' '.join(tokens)

    return cleaned_text

In [157]:
transcript = list(data["Transcript"])
video_title = list(data["Video Title"])

In [158]:
import nltk
nltk.download('omw-1.4')

cleaned_transcript_ = []
cleaned_video_title_ = []

similarity_score = []
for i in range(len(data)):
    cleaned_transcript = clean_and_preprocess(transcript[i])
    cleaned_video_title = clean_and_preprocess(video_title[i])

    cleaned_transcript_.append(cleaned_transcript)
    cleaned_video_title_.append(cleaned_video_title)

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/codyoshiro/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [159]:
# Create a TF-IDF vectorizer
vectorizer = TfidfVectorizer(max_features=1000, stop_words='english')

# Fit and transform the TF-IDF vectorizer on the combined text data
tfidf_matrix = vectorizer.fit_transform(cleaned_video_title_ + cleaned_transcript_)

In [160]:
# Perform Latent Semantic Analysis (LSA) using Truncated SVD
num_topics = 1  # Adjust the number of topics as needed
lsa = TruncatedSVD(n_components=num_topics, random_state=42)
lsa_matrix = lsa.fit_transform(tfidf_matrix)

In [161]:
# Create DataFrames to display the results for titles and transcripts
results_titles = pd.DataFrame({'Video Title': cleaned_video_title_})
for topic_idx in range(num_topics):
    results_titles[f'Topic {topic_idx + 1}'] = lsa_matrix[:len(cleaned_video_title_), topic_idx]

results_transcripts = pd.DataFrame({'Transcript': cleaned_transcript_})
for topic_idx in range(num_topics):
    results_transcripts[f'Topic {topic_idx + 1}'] = lsa_matrix[len(cleaned_transcript_):, topic_idx]

In [162]:
results_titles

,Video Title,Topic 1
0,got hunted fbi,0.203294
1,real life giant wont believe exist,0.049437
2,mythical creature actually existed,0.037122
3,real giant caught camera,0.039735
4,real life giant,0.041613
...,...,...
150,building unbreakable tesseract cube,0.008812
151,lanky kong listens dk rap first time,0.093379
152,become american ninja warrior daequan hamlinz,0.000000
153,anecdote trump data,0.000000


In [163]:
results_transcripts

,Transcript,Topic 1
0,tied fbi agent word popif steal hundred thousa...,0.429805
1,hey whats happening yall welcome backto anothe...,0.276514
2,used taste daddy doesntlove good story especia...,0.257945
3,hi said giant imaginary onlyexist fairy tale m...,0.252550
4,musichey guy whats charlie andtoday going look...,0.386751
...,...,...
150,course main priority make box unbreakablebut a...,0.569472
151,yo guyslanky kong another reaction videowe got...,0.437942
152,rightwhats guy welcome doom andninja warrior c...,0.675518
153,critic tv show mythbusters claimthat show misr...,0.369441


In [164]:
label_list = []
for i in range(len(scores_df)):
  if results_titles["Topic 1"][i] >= 0.25 or results_transcripts["Topic 1"][i] >= 0.25:
    label_list.append(1)
  else:
    label_list.append(0)

In [165]:
label_list

[1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1]

In [166]:
true_labels = data["class"]

In [167]:
True_clickbait = 0
False_clickbait = 0

True_nonclickbait = 0
False_nonclickbiat = 0

for i in range(len(true_labels)):
  if label_list[i] == 1 and  true_labels[i] == 1:
    True_clickbait += 1
  elif label_list[i] == 1 and  true_labels[i] == 0:
    False_clickbait += 1
  elif label_list[i] == 0 and  true_labels[i] == 1:
    False_nonclickbiat += 1
  elif label_list[i] == 0 and  true_labels[i] == 0:
    True_nonclickbait += 1

In [168]:
print("True_clickbait", True_clickbait)
print("False_clickbait", False_clickbait)

print("True_nonclickbait", True_nonclickbait)
print("False_nonclickbiat", False_nonclickbiat)

True_clickbait 61
False_clickbait 70
True_nonclickbait 14
False_nonclickbiat 10


In [169]:
accuracy = (True_clickbait + True_nonclickbait) / (True_clickbait + False_clickbait + True_nonclickbait + False_nonclickbiat)
accuracy

0.4838709677419355

In [170]:
Precision = True_clickbait / (True_clickbait + False_clickbait)
Precision

0.46564885496183206

In [171]:
Recall = True_clickbait / (True_clickbait + False_nonclickbiat)
Recall

0.8591549295774648

In [172]:
#creating data frame with video title and scores
results_titles_scores = pd.DataFrame({'Video Title': cleaned_video_title_})
for topic_idx in range(num_topics):
    results_titles_scores[f'Topic {topic_idx + 1}'] = lsa_matrix[:len(cleaned_video_title_), topic_idx]

#creating data frame with transcipt and scores
results_transcripts_scores = pd.DataFrame({'Transcript': cleaned_transcript_})
for topic_idx in range(num_topics):
    results_transcripts_scores[f'Topic {topic_idx + 1}'] = lsa_matrix[len(cleaned_transcript_):, topic_idx]

scores_df = pd.DataFrame(data['class'])

for topic_idx in range(num_topics):
    scores_df['Title Scores'] = lsa_matrix[:len(cleaned_video_title_), topic_idx]
    scores_df['Transcript Scores'] = lsa_matrix[len(cleaned_transcript_):, topic_idx]
    scores_df['Views'] = data['Views']
    scores_df['Likes'] = data['Likes']
    scores_df['Dislikes'] = data['Dislikes']
    scores_df['LSA Feature'] = 0
    

scores_df

,class,Title Scores,Transcript Scores,Views,Likes,Dislikes,LSA Feature
0,1,0.203294,0.429805,42724724,2005151,24646,0
1,1,0.049437,0.276514,6890718,15222,2858,0
2,1,0.037122,0.257945,2089601,46750,1954,0
3,1,0.039735,0.252550,518170,4033,271,0
4,1,0.041613,0.386751,9244943,64025,6508,0
...,...,...,...,...,...,...,...
150,0,0.008812,0.569472,1687124,81479,1474,0
151,0,0.093379,0.437942,839745,102699,567,0
152,0,0.000000,0.675518,819705,68459,402,0
153,0,0.000000,0.369441,980055,36060,1347,0


In [173]:
for i in range(len(scores_df)):
  if results_titles["Topic 1"][i] >= 0.001 or results_transcripts["Topic 1"][i] >= 0.001:
    scores_df["LSA Feature"].iloc[i] = 1
  else:
    scores_df["LSA Feature"].iloc[i] = 0
    
scores_df = scores_df.drop(columns = ["Title Scores", "Transcript Scores"])
scores_df

/Users/codyoshiro/opt/anaconda3/lib/python3.9/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


,class,Views,Likes,Dislikes,LSA Feature
0,1,42724724,2005151,24646,1
1,1,6890718,15222,2858,1
2,1,2089601,46750,1954,1
3,1,518170,4033,271,1
4,1,9244943,64025,6508,1
...,...,...,...,...,...
150,0,1687124,81479,1474,1
151,0,839745,102699,567,1
152,0,819705,68459,402,1
153,0,980055,36060,1347,1


In [174]:
scores_df["LSA Feature"].value_counts()

1    155
Name: LSA Feature, dtype: int64

In [175]:
scores_df

,class,Views,Likes,Dislikes,LSA Feature
0,1,42724724,2005151,24646,1
1,1,6890718,15222,2858,1
2,1,2089601,46750,1954,1
3,1,518170,4033,271,1
4,1,9244943,64025,6508,1
...,...,...,...,...,...
150,0,1687124,81479,1474,1
151,0,839745,102699,567,1
152,0,819705,68459,402,1
153,0,980055,36060,1347,1


In [176]:
#SVM Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn import metrics

#Logistic Regression Imports
from sklearn import linear_model

#KNN Imports
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.model_selection import train_test_split 

#Naive Bayes Imports
from sklearn.naive_bayes import GaussianNB

#Gaussian Process Regression
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF

#pre: df_xcols and ydf_ycol are columns of a data frame.
#if inserting multiple cols to df_xcols, enter parameter as 'col1, col2'
#df is a dataframe
#df_ycols is target column of data frame
def train_svm(df, df_xcols, df_ycol, kernel_type):
    x_svm = df[df_xcols]
    y_svm = df[df_ycol]
    # Split dataset into training set and test set
    X_train, X_test, y_train, y_test = train_test_split(x_svm, y_svm, test_size=0.3) # 70% training and 30% test

    clf = SVC(kernel=kernel_type)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    
    # Model Accuracy: how often is the classifier correct?
    print("Accuracy:",metrics.accuracy_score(y_test, y_pred))
    
    #Model Precision
    print("Precision:",metrics.precision_score(y_test, y_pred))

    #Model Recall
    print("Recall:",metrics.recall_score(y_test, y_pred))
    
    #Model F1 Score
    print("F1 Score: ", metrics.f1_score(y_test, y_pred))
    
    return clf
    
def train_logr(df, feature_cols, df_ycol):
    x_logr = df[feature_cols]
    y_logr = df[df_ycol]
    #split dataset into training set and test set
    X_train, X_test, y_train, y_test = train_test_split(x_logr, y_logr, test_size=0.3) # 70% training and 30% test

    logr = linear_model.LogisticRegression(max_iter = 1000)
    logr.fit(X_train, y_train.values.ravel())
    y_pred = logr.predict(X_test)
    
    #Model Accuracy
    print("Accuracy:",metrics.accuracy_score(y_test, y_pred))
    
    #Model Precision
    print("Precision:",metrics.precision_score(y_test, y_pred))

    #Model Recall
    print("Recall:",metrics.recall_score(y_test, y_pred))
    
    #Model F1 Score
    print("F1 Score: ", metrics.f1_score(y_test, y_pred))
    
    return logr

def train_KNN(df, feature_cols, df_ycol, neighbors):

    # Create feature and target arrays 
    x_KNN = df[feature_cols]
    y_KNN = df[df_ycol]

    # Split into training and test set 
    X_train, X_test, y_train, y_test = train_test_split(x_KNN, y_KNN, test_size = 0.3) 

    knn = KNeighborsClassifier(n_neighbors=neighbors) 

    knn.fit(X_train, y_train) 

    # Predict on dataset which model has not seen before 
    y_pred = knn.predict(X_test)
    
    #Model Accuracy
    print("Accuracy:",metrics.accuracy_score(y_test, y_pred))
    
    #Model Precision
    print("Precision:",metrics.precision_score(y_test, y_pred))

    #Model Recall
    print("Recall:",metrics.recall_score(y_test, y_pred))
    
    #Model F1 Score
    print("F1 Score: ", metrics.f1_score(y_test, y_pred))
    
    return knn

In [177]:
#Below are the models trained on the features w/o topic modeling techniques applied
model = train_svm(scores_df, ['Views', 'Likes', 'Dislikes'], 'class', 'rbf')

Accuracy: 0.6595744680851063
Precision: 0.8333333333333334
Recall: 0.4166666666666667
F1 Score:  0.5555555555555556


In [178]:
model = train_logr(scores_df, ['Views', 'Likes', 'Dislikes'], 'class')

Accuracy: 0.8297872340425532
Precision: 0.85
Recall: 0.7727272727272727
F1 Score:  0.8095238095238095


In [179]:
model = train_KNN(scores_df, ['Views', 'Likes', 'Dislikes'], 'class', 7)

Accuracy: 0.723404255319149
Precision: 0.6521739130434783
Recall: 0.75
F1 Score:  0.6976744186046512


In [180]:
#trained with LSA Feauture

In [181]:
model = train_logr(scores_df, ['Views', 'Likes', 'Dislikes', 'LSA Feature'], 'class')
#adding the new features here minorly improves accuracy by ~2% but significantly improves recall and F1 score.

Accuracy: 0.7872340425531915
Precision: 0.8333333333333334
Recall: 0.6818181818181818
F1 Score:  0.75


In [182]:
model = train_KNN(scores_df, ['Views', 'Likes', 'Dislikes', 'LSA Feature'], 'class', 7)

Accuracy: 0.8085106382978723
Precision: 0.8260869565217391
Recall: 0.7916666666666666
F1 Score:  0.8085106382978723


In [188]:
model = train_svm(scores_df, ['Views', 'Likes', 'Dislikes', 'LSA Feature'], 'class', 'rbf')

Accuracy: 0.851063829787234
Precision: 1.0
Recall: 0.6956521739130435
F1 Score:  0.8205128205128205
